In [5]:
import os
os.chdir('D:/Python/Flies/Yinhe')
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

class triply_filter:
    def __init__(self, stock):
        self.stock = stock
        self.data_df = self.get_csv_data()
    def get_csv_data(self):
        """
        给定股票，获取给股票2020-10-01之后的分钟数据：开盘价，收盘价，最高价，最低价
        在获取macd后截去前面的数据，只保留2021-03-01之后的数据
        数据来自joinquant，从2020/1/2开盘到2021/12/30收盘
        open_df等csv文件dataframe列为stock代码，行为分钟级时间，value为开盘价
        最后生成的data_df为某只股票的分钟级数据，列为开盘收盘最高最低价，以及macd数据，行为分钟级数据，value为对应的值
        """
        stock = self.stock
        open_df = pd.read_csv('dataset/KC50/open_sum.csv').set_index('date')
        close_df = pd.read_csv('dataset/KC50/close_sum.csv').set_index('date')
        high_df = pd.read_csv('dataset/KC50/high_sum.csv').set_index('date')
        low_df = pd.read_csv('dataset/KC50/low_sum.csv').set_index('date')
        open_df.index = pd.to_datetime(open_df.index)
        close_df.index = pd.to_datetime(close_df.index)
        high_df.index = pd.to_datetime(high_df.index)
        low_df.index = pd.to_datetime(low_df.index)
        data_df = pd.DataFrame(index = close_df.index)
        data_df['open'] = open_df[stock]
        data_df['close'] = close_df[stock]
        data_df['high'] = high_df[stock]
        data_df['low'] = low_df[stock]
        data_df = data_df[data_df.index >= pd.to_datetime('2020-10-01')]
        data_df['date_date'] = data_df.index.date
        
        def macd_generator(time_delta) : 
            """
            由收盘价获取不同时间周期time_delta下的macd数据
            首先计算EMA指数平均数指标，再通过EMA计算MACD
            生成新的MACD列添加到data_df中
            由于EMA的计算需要用到过去的EMA数据，
            这里从2020-10-01开始计算EMA并会在最后截取2021-03-01之后的结果以保证尽可能小的消除初始值的影响
            input: 时间周期time_delta，由于60d和15d两个滤网需要用到macd数据，这里time_delta在之后会取60和15
            """
            data_df['ema_30'] = data_df['close']
            data_df['ema_50'] = data_df['close']
            data_df['ema_9'] = data_df['close']
            for i in range(time_delta, len(data_df)) : 
                data_df.loc[data_df.index[i],'ema_30'] = (2*data_df['close'].iloc[i] + (30-1)*data_df['ema_30'].iloc[i-time_delta]) / (30+1)
                data_df.loc[data_df.index[i],'ema_50'] = (2*data_df['close'].iloc[i] + (50-1)*data_df['ema_50'].iloc[i-time_delta]) / (50+1)
                data_df.loc[data_df.index[i],'ema_9'] = (2*data_df['close'].iloc[i] + (9-1)*data_df['ema_9'].iloc[i-time_delta]) / (9+1)
            a,b,c = 9,30,50    
            data_df['macd_'+str(time_delta)] = (-(a-1)/(a-b) + (a-1)/(a-c))*2 * data_df['ema_9'] + \
                2*(1 + (b-1)/(a-b)) * data_df['ema_30'] + 2*(-1 - (c-1)/(a-c)) * data_df['ema_50']   
        macd_generator(60)
        macd_generator(15)
        data_df = data_df[data_df.index >= pd.to_datetime('2021-03-01')] # 取2021-03-01之后的结果以保证尽可能小的消除EMA初始值的影响
        return data_df
    
    def get_data(self, query_time, time_delta, type_ = 'close'):
        """
        func: 仿照箱线图，给定一个时间段，获取该时间段内的开盘收盘价，最高最低价，
            从而起到获取不同时间周期下的开盘收盘价，最高最低价的作用。
            给定股票，开始时间，时间间隔，返回过去time_delta分钟里的close,open,high,low
        input: query_time: '2016-01-04 09:32:00', 
            time_delta: int
        """
        if type_ == 'open' : 
            return self.data_df.loc[query_time - pd.Timedelta(str(time_delta - 1) + 'minutes'), 'open']
        if type_ == 'close' : 
            return self.data_df.loc[query_time, 'close']
        if type_ == 'high' : 
            return self.data_df.loc[query_time - pd.Timedelta(str(time_delta - 1) + 'minutes'): query_time, 'high'].max()
        if type_ == 'low' : 
            return self.data_df.loc[query_time - pd.Timedelta(str(time_delta - 1) + 'minutes'): query_time, 'low'].min()    
    def signal_1(self) : 
        """
        第一个信号指标的计算。
        在60分钟上，计算MACD(30,50,9)，大于0则BK1=1、SK1=0；否则BK1=0、SK1=1。
        由于BK和SK成对出现，这里只考虑BK，之后的signal方法同理
        """
        signal_df = pd.DataFrame(self.data_df.macd_60 , index = self.data_df.index)
        signal_df['bk1'] = None
        signal_df.loc[signal_df['macd_60']>0,'bk1'] = 1
        signal_df.loc[signal_df['macd_60']<0,'bk1'] = 0
        return signal_df[['bk1']]

    def signal_2(self): 
        """
        第二个信号指标的计算。
        在15分钟上，若最近一次为底分型，其右K线收盘价高于前一次底分型的右K线收盘价（股价不创新低），
        或虽然其右K线收盘价低于前一次底分型的右K线收盘价，但MACD(30,50,9)柱线高于上次（底背离），则BK2=1、SK2=0；
        反之若最近一次为顶分型，其右K线收盘价低于前一次顶分型的右K线收盘价（股价不创新高），
        或虽然其右K线收盘价高于前一次顶分型的右K线收盘价，但MACD(30,50,9)柱线低于上次（顶背离），则BK2=0、SK2=1。
        """
        signal_df = pd.DataFrame(self.data_df.macd_15 , index = self.data_df.index)
        signal_df['date'] = signal_df.index
        signal_df['close'] = signal_df['date'].apply(self.get_data, args = (15,'close'))
        signal_df['high'] = signal_df['date'].apply(self.get_data, args = (15,'high'))
        signal_df['low'] = signal_df['date'].apply(self.get_data, args = (15,'low'))

        signal_df['type'] = None
        for i in range(15,len(signal_df)) : 
            if signal_df['high'][i-15] > signal_df['high'][i] and signal_df['high'][i-15] > signal_df['high'][i-30] and\
                       signal_df['low'][i-15] > signal_df['low'][i] and signal_df['low'][i-15] > signal_df['low'][i-30] :

                signal_df.loc[signal_df.index[i],'type'] = 'top'

            elif signal_df['high'][i-15] < signal_df['high'][i] and signal_df['high'][i-15] < signal_df['high'][i-30] and\
                         signal_df['low'][i-15] < signal_df['low'][i] and signal_df['low'][i-15] < signal_df['low'][i-30] :

                signal_df.loc[signal_df.index[i],'type'] = 'bot'

        signal_df = signal_df.reset_index(drop=True)
        signal_df['bk2'] = None
        for i in range(15) : 

            tmp_df = signal_df[signal_df.index % 15 == i]
            tmp_df = tmp_df.copy()

            bot_df = tmp_df[tmp_df['type']=='bot']
            buy_df = bot_df[(bot_df['close'].diff()>0) | (bot_df['macd_15'].diff()>0)]
            tmp_df.loc[buy_df.index, 'bk2'] = 1

            bot_df = tmp_df[tmp_df['type']=='top']
            sell_df = bot_df[(bot_df['close'].diff()<0) | (bot_df['macd_15'].diff()<0)]
            tmp_df.loc[sell_df.index, 'bk2'] = 0

            tmp_df['bk2'] = tmp_df['bk2'].fillna(method='ffill')


            signal_df.loc[tmp_df.index,'bk2'] = tmp_df['bk2']
        signal_df = signal_df.set_index('date')
        return signal_df[['bk2']]

    def signal_3(self, m):
        """
        第三个信号指标的计算。
        在4分钟上，若最高价高于前一根K线最高价*(1+常数参数m)，则BK3=1、SK3=0；
        若最低价低于前一根K线最低价-常数参数m，则BK3=0、SK3=1
        """
        signal_df = pd.DataFrame(self.data_df.macd_15 , index = self.data_df.index)
        signal_df['date'] = signal_df.index
        signal_df['high'] = signal_df['date'].apply(self.get_data, args = (4,'high'))
        signal_df['low'] = signal_df['date'].apply(self.get_data, args = (4,'low'))

        signal_df['bk3'] = None

        def inner_func(i,m):
            i = signal_df.index.get_loc(i)
            if signal_df['high'][i] > signal_df['high'][i-4]*(1 + m) :   # 这里修改为和上次股价的相对涨幅
                return 1
            elif signal_df['low'][i] < signal_df['low'][i-4]*(1 - m) :
                return 0
        signal_df['bk3'] = signal_df['date'].apply(inner_func, args = (m,))

        return signal_df[['bk3']]
    
    def train_func(self, time_begin = '2021-04-01', time_end = '2021-11-01') : 
        """
        对4月到10月的历史数据进行回测，找出最优的m参数值，
        函数整体与trade_func比较接近
        input: time_begin/time_end: 回测开始时间/结束时间，形如'2020-01-01' str
        """
        m_list = []
        signal_df_1 = self.signal_1()
        signal_df_2 = self.signal_2()
        test_list = [0, 0.002, 0.004, 0.006, 0.008, 0.012, 0.014, 0.016, 0.018]
        for m in  test_list: 
            signal_df_3 = self.signal_3(m)
            # 将三种信号合并到一张dataframe上，对于bk全为1的设置交易信号为买入1，反之为卖出信号0
            signal_df = pd.concat([signal_df_1,signal_df_2,signal_df_3], axis=1)
            buy_df = signal_df[(signal_df['bk1']==1)&(signal_df['bk2']==1)&(signal_df['bk3']==1)]
            sell_df = signal_df[(signal_df['bk1']==0)&(signal_df['bk2']==0)&(signal_df['bk3']==0)]
            capacity = pd.DataFrame(index = signal_df.index)
            capacity['signal'] = 0
            capacity.loc[buy_df.index,'signal'] = 1
            capacity.loc[sell_df.index,'signal'] = -1
            capacity = capacity[capacity.index >= pd.to_datetime(time_begin)] # 只选用2021-04-01起的数据
            capacity = capacity[capacity.index <= pd.to_datetime(time_end)] # 只选用2021-11-01终了的数据，留出m的验证集数据
            # 构建交易回测记录，每min循环，每日平仓
            capacity['close'] = self.data_df['close']
            capacity['cash'] = 0   # 现金
            capacity['stock_shares'] = 0   # 持有股份数
            capacity['delta_change'] = 0   # 计算胜率用的过程量，初始为0
            capacity['total_capacity'] = 0   # 持股现值+现金
            capacity['date_date'] = capacity.index.date

            grouped = capacity.groupby(by='date_date')
            for key, daily_df in grouped:
                # 每次买卖一手股票
                daily_df['stock_shares'] = 1 * daily_df['signal'] # signal为1时做多，股票数加1；反之减1
                daily_df['cash'] = - daily_df['stock_shares'] * daily_df['close'] # cash变动数为买卖股票数*当前股价，变动方向和股票数方向相反
                daily_df['delta_change'] = daily_df['stock_shares'] * daily_df.iloc[-1]['close'] + daily_df['cash'] 
                # 这笔交易的盈亏取决于当日收盘时平仓情况，记录下这笔交易的盈亏为delta_change

                # 每日平仓
                daily_df.loc[daily_df.index[-1], 'stock_shares'] = - daily_df.loc[daily_df.index[:-1], 'stock_shares'].sum()
                daily_df.loc[daily_df.index[-1], 'cash'] = - daily_df.loc[daily_df.index[-1], 'stock_shares'] * daily_df.loc[daily_df.index[-1], 'close']

                capacity.loc[daily_df.index] = daily_df

            # 将数据cumsum，从变化量调整为总量
            capacity['stock_shares'] = capacity['stock_shares'].cumsum()
            capacity['cash'] = capacity['cash'].cumsum()
            # 设置初始cash和shares
            capacity['stock_shares'] = capacity['stock_shares'] + 500
            capacity['cash'] = capacity['cash'] + 5000
            capacity['total_capacity'] = capacity['cash'] + capacity['stock_shares'] * capacity['close']
            
            m_list.append(capacity['total_capacity'][-1])
        return test_list[m_list.index(max(m_list))]
    
    def trade_func(self, m, time_begin = '2021-11-01', time_end = '2022-01-01'):
        """
        在获取4月到10月间的最优的m参数值后，以它为参数开始计算交易11月到12月的盈亏
        input: m: 需要输入的参数m，可以来自于train_func函数所找到的过去最优m值，也可以手动输入
            time_begin/time_end: 回测开始时间/结束时间，形如'2020-01-01' str
        """

        # 生成60min，15min，4min三种周期下的信号
        signal_df_1 = self.signal_1()
        signal_df_2 = self.signal_2()
        signal_df_3 = self.signal_3(m)
        print(len(signal_df_3[signal_df_3['bk3']==1])/len(signal_df_3))
        # 将三种信号合并到一张dataframe上，对于bk全为1的设置交易信号为买入1，反之为卖出信号0
        signal_df = pd.concat([signal_df_1,signal_df_2,signal_df_3], axis=1)
        buy_df = signal_df[(signal_df['bk1']==1)&(signal_df['bk2']==1)&(signal_df['bk3']==1)]
        sell_df = signal_df[(signal_df['bk1']==0)&(signal_df['bk2']==0)&(signal_df['bk3']==0)]
        capacity = pd.DataFrame(index = signal_df.index)
        capacity['signal'] = 0
        capacity.loc[buy_df.index,'signal'] = 1
        capacity.loc[sell_df.index,'signal'] = -1
        capacity = capacity[capacity.index >= pd.to_datetime(time_begin)] # 只选用time_begin = '2021-11-01'起的数据
        capacity = capacity[capacity.index <= pd.to_datetime(time_end)] # 只选用time_end = '2022-01-01'起的数据

        # 构建交易回测记录，每min循环，每日平仓
        capacity['close'] = self.data_df['close']
        capacity['cash'] = 0   # 现金
        capacity['stock_shares'] = 0   # 持有股份数
        capacity['delta_change'] = 0   # 计算胜率用的过程量，初始为0
        capacity['total_capacity'] = 0   # 持股现值+现金
        capacity['date_date'] = capacity.index.date


        grouped = capacity.groupby(by='date_date')
        for key, daily_df in grouped:
            # 每次买卖一手股票
            daily_df['stock_shares'] = 1 * daily_df['signal'] # signal为1时做多，股票数加1；反之减1
            daily_df['cash'] = - daily_df['stock_shares'] * daily_df['close'] # cash变动数为买卖股票数*当前股价，变动方向和股票数方向相反
            daily_df['delta_change'] = daily_df['stock_shares'] * daily_df.iloc[-1]['close'] + daily_df['cash'] 
            # 这笔交易的盈亏取决于当日收盘时平仓情况，记录下这笔交易的盈亏为delta_change

            # 每日平仓
            daily_df.loc[daily_df.index[-1], 'stock_shares'] = - daily_df.loc[daily_df.index[:-1], 'stock_shares'].sum()
            daily_df.loc[daily_df.index[-1], 'cash'] = - daily_df.loc[daily_df.index[-1], 'stock_shares'] * daily_df.loc[daily_df.index[-1], 'close']

            capacity.loc[daily_df.index] = daily_df

        # 将数据cumsum，从变化量调整为总量
        capacity['stock_shares'] = capacity['stock_shares'].cumsum()
        capacity['cash'] = capacity['cash'].cumsum()
        # 设置初始cash和shares
        capacity['stock_shares'] = capacity['stock_shares'] + 500
        capacity['cash'] = capacity['cash'] + 5000
        capacity['total_capacity'] = capacity['cash'] + capacity['stock_shares'] * capacity['close']

        # 计算胜率
        win = len(capacity[capacity['delta_change']>0])
        lose = len(capacity[capacity['delta_change']<0])
        rate = win/(win+lose)
#         rate = (win+lose)
        print('胜率：',rate)

        print('stock:',stock,'m=',m)
        capacity['total_capacity'][:-2].plot()
        plt.title(stock+' with m = '+str(m))
        plt.xlabel("Time")
        plt.ylabel("Capital")
        plt.show()
        return rate, capacity[['signal','close','cash','stock_shares','total_capacity']]
    
if __name__ == '__main__':
    stock = '688005.XSHG'
    aa = triply_filter(stock)
#     best_m = aa.train_func()  # 从0 到 0.08 中检索最优的参数m值，可以不运行此函数直接将想要的参数m值代入下面的trade_func函数中
    trade_list = aa.trade_func(m = 0.6, time_begin = '2021-04-01')